# Fine-tune a pretrained model

1. [Prepare a dataset](#prepare-a-dataset)
2. [Train](#train)

    2.1. [with PyTorch Trainer](#1-train-with-pytorch-trainer)

## Prepare a dataset

Begin by loading the Yelp Reviews dataset. You need a tokenizer to process the text.


In [1]:
from datasets import load_dataset

dataset = load_dataset('yelp_review_full')
dataset['train'][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

def tokenize_function(examples):
    return tokenizer(examples['text'],padding='max_length',truncation=True)

tokenized_datasets = dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

You can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes

In [4]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## Train

### 1. Train with PyTorch Trainer

Huggingface Transformers provide a Trainer class optimized for training Huggingface Transformers model. Start by loading your model and specify the number of expected labels. From the Yelp Review dataset card, you know there are five labels


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Training hyperparameters

Create a TrainingArguments class which contains all the hyperparameters you can tune as well as flags for activating different options. If you'd like to monitor your evaluation metrics during fine-tuning, specify the evaluation_strategy parameter to report the evaluation metric at the end of each epoch.

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir = 'test_trainer', evaluation_strategy="epoch")

#### Evaluate

In [7]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,
)

In [9]:
trainer.train()

: 

: 